
## Overview

This notebook shows you how to create and query a table or DataFrame loaded from data stored in AWS S3. There are two ways to establish access to S3: [IAM roles](https://docs.databricks.com/user-guide/cloud-configurations/aws/iam-roles.html) and access keys.

*We recommend using IAM roles to specify which cluster can access which buckets. Keys can show up in logs and table metadata and are therefore fundamentally insecure.* If you do use keys, you'll have to escape the `/` in your keys with `%2F`.

This is a **Python** notebook so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` magic command. Python, Scala, SQL, and R are all supported.

Lee el archivo y cárgalo en una vista temporal usando Pyspark

In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [0]:
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.2 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
%fs ls /FileStore/tables/

path,name,size,modificationTime
dbfs:/FileStore/tables/Mall_Customers.csv,Mall_Customers.csv,4286,1729466845000


In [0]:
%fs rm /FileStore/tables/Mall_Customers-4.csv

res5: Boolean = true

a. Realiza al menos 4 consultas, ejecutando la función sprak.sql

In [0]:
# Leer el archivo CSV desde /FileStore/tables/ usando Spark
df = spark.read.csv('/FileStore/tables/Mall_Customers.csv', header=True, inferSchema=True)

# Mostrar las primeras filas del DataFrame
df.show()

+----------+------+---+------------------+----------------------+
|CustomerID| Genre|Age|Annual Income (k$)|Spending Score (1-100)|
+----------+------+---+------------------+----------------------+
|         1|  Male| 19|                15|                    39|
|         2|  Male| 21|                15|                    81|
|         3|Female| 20|                16|                     6|
|         4|Female| 23|                16|                    77|
|         5|Female| 31|                17|                    40|
|         6|Female| 22|                17|                    76|
|         7|Female| 35|                18|                     6|
|         8|Female| 23|                18|                    94|
|         9|  Male| 64|                19|                     3|
|        10|Female| 30|                19|                    72|
|        11|  Male| 67|                19|                    14|
|        12|Female| 35|                19|                    99|
|        1

In [0]:
df.createOrReplaceTempView("mi_tabla_csv")

In [0]:
spark.sql("SELECT * FROM mi_tabla_csv").show()

+----------+------+---+------------------+----------------------+
|CustomerID| Genre|Age|Annual Income (k$)|Spending Score (1-100)|
+----------+------+---+------------------+----------------------+
|         1|  Male| 19|                15|                    39|
|         2|  Male| 21|                15|                    81|
|         3|Female| 20|                16|                     6|
|         4|Female| 23|                16|                    77|
|         5|Female| 31|                17|                    40|
|         6|Female| 22|                17|                    76|
|         7|Female| 35|                18|                     6|
|         8|Female| 23|                18|                    94|
|         9|  Male| 64|                19|                     3|
|        10|Female| 30|                19|                    72|
|        11|  Male| 67|                19|                    14|
|        12|Female| 35|                19|                    99|
|        1

In [0]:
spark.sql("SELECT * FROM mi_tabla_csv WHERE Age > 30").show()

+----------+------+---+------------------+----------------------+
|CustomerID| Genre|Age|Annual Income (k$)|Spending Score (1-100)|
+----------+------+---+------------------+----------------------+
|         5|Female| 31|                17|                    40|
|         7|Female| 35|                18|                     6|
|         9|  Male| 64|                19|                     3|
|        11|  Male| 67|                19|                    14|
|        12|Female| 35|                19|                    99|
|        13|Female| 58|                20|                    15|
|        15|  Male| 37|                20|                    13|
|        17|Female| 35|                21|                    35|
|        19|  Male| 52|                23|                    29|
|        20|Female| 35|                23|                    98|
|        21|  Male| 35|                24|                    35|
|        23|Female| 46|                25|                     5|
|        2

In [0]:
spark.sql("SELECT * FROM mi_tabla_csv WHERE `Annual Income (k$)`>20 AND Genre = 'Male'").show()


+----------+-----+---+------------------+----------------------+
|CustomerID|Genre|Age|Annual Income (k$)|Spending Score (1-100)|
+----------+-----+---+------------------+----------------------+
|        18| Male| 20|                21|                    66|
|        19| Male| 52|                23|                    29|
|        21| Male| 35|                24|                    35|
|        22| Male| 25|                24|                    73|
|        24| Male| 31|                25|                    73|
|        26| Male| 29|                28|                    82|
|        28| Male| 35|                28|                    61|
|        31| Male| 60|                30|                     4|
|        33| Male| 53|                33|                     4|
|        34| Male| 18|                33|                    92|
|        42| Male| 24|                38|                    92|
|        43| Male| 48|                39|                    36|
|        52| Male| 33|   

In [0]:
spark.sql("SELECT * FROM mi_tabla_csv WHERE `Spending Score (1-100)`>30 AND Genre = 'Female'").show()


+----------+------+---+------------------+----------------------+
|CustomerID| Genre|Age|Annual Income (k$)|Spending Score (1-100)|
+----------+------+---+------------------+----------------------+
|         4|Female| 23|                16|                    77|
|         5|Female| 31|                17|                    40|
|         6|Female| 22|                17|                    76|
|         8|Female| 23|                18|                    94|
|        10|Female| 30|                19|                    72|
|        12|Female| 35|                19|                    99|
|        14|Female| 24|                20|                    77|
|        17|Female| 35|                21|                    35|
|        20|Female| 35|                23|                    98|
|        27|Female| 45|                28|                    32|
|        29|Female| 40|                29|                    31|
|        30|Female| 23|                29|                    87|
|        3

In [0]:
%sql
SELECT * FROM mi_tabla_csv

CustomerID,Genre,Age,Annual Income (k$),Spending Score (1-100)
1,Male,19,15,39
2,Male,21,15,81
3,Female,20,16,6
4,Female,23,16,77
5,Female,31,17,40
6,Female,22,17,76
7,Female,35,18,6
8,Female,23,18,94
9,Male,64,19,3
10,Female,30,19,72


4. Utiliza el modo SQL del notebook,

In [0]:
%sql
SELECT * FROM mi_tabla_csv WHERE Genre = 'Male'

CustomerID,Genre,Age,Annual Income (k$),Spending Score (1-100)
1,Male,19,15,39
2,Male,21,15,81
9,Male,64,19,3
11,Male,67,19,14
15,Male,37,20,13
16,Male,22,20,79
18,Male,20,21,66
19,Male,52,23,29
21,Male,35,24,35
22,Male,25,24,73


a. Realiza al menos 4 consultas, ejecutando.

1. Filtrar clientes de género masculino (Male) con ingresos superiores a 70k:

In [0]:
%sql
SELECT CustomerID, Age, `Annual Income (k$)`, `Spending Score (1-100)` 
FROM mi_tabla_csv 
WHERE Genre = 'Male' AND `Annual Income (k$)` > 70

CustomerID,Age,Annual Income (k$),Spending Score (1-100)
127,43,71,35
128,40,71,95
129,59,71,11
130,38,71,75
131,47,71,9
132,39,71,75
135,20,73,5
138,32,73,73
139,19,74,10
142,32,75,93


2. Obtener el ingreso promedio y el puntaje de gasto promedio por género:

In [0]:
%sql
SELECT Genre, AVG(`Annual Income (k$)`) AS Ingreso_Promedio, AVG(`Spending Score (1-100)`) AS Puntaje_Promedio 
FROM mi_tabla_csv 
GROUP BY Genre

Genre,Ingreso_Promedio,Puntaje_Promedio
Female,59.25,51.526785714285715
Male,62.22727272727273,48.51136363636363


3. Ordenar los clientes por el puntaje de gasto en orden descendente:

In [0]:
%sql
SELECT CustomerID, Age, `Spending Score (1-100)` 
FROM mi_tabla_csv 
ORDER BY `Spending Score (1-100)` DESC

CustomerID,Age,Spending Score (1-100)
12,35,99
20,35,98
146,28,97
186,30,97
128,40,95
168,33,95
8,23,94
142,32,93
164,31,93
34,18,92


4. Obtener los clientes con el mayor ingreso anual y su respectivo puntaje de gasto:

In [0]:
%sql
SELECT CustomerID, `Annual Income (k$)`, `Spending Score (1-100)` 
FROM mi_tabla_csv 
ORDER BY `Annual Income (k$)` DESC 
LIMIT 5

CustomerID,Annual Income (k$),Spending Score (1-100)
200,137,83
199,137,18
197,126,28
198,126,74
195,120,16


b. Crea una tabla temporal mediante la inserción de los datos de la vista temporal

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW mi_vista_temporal AS
SELECT * FROM mi_tabla_csv

In [0]:
%sql
SELECT * FROM mi_vista_temporal

CustomerID,Genre,Age,Annual Income (k$),Spending Score (1-100)
1,Male,19,15,39
2,Male,21,15,81
3,Female,20,16,6
4,Female,23,16,77
5,Female,31,17,40
6,Female,22,17,76
7,Female,35,18,6
8,Female,23,18,94
9,Male,64,19,3
10,Female,30,19,72


Archivo parquet

In [0]:
# Ejecutar la consulta SQL y almacenar el resultado en un DataFrame
df_resultado = spark.sql("""SELECT * FROM mi_tabla_csv """)

In [0]:
# Guardar el resultado en formato Parquet en la carpeta especificada
df_resultado.write.parquet("/FileStore/tables/resultado_parquet")

In [0]:
%fs ls /FileStore/tables/resultado_parquet

path,name,size,modificationTime
dbfs:/FileStore/tables/resultado_parquet/_SUCCESS,_SUCCESS,0,1729476572000
dbfs:/FileStore/tables/resultado_parquet/_committed_1288630687672049619,_committed_1288630687672049619,123,1729476572000
dbfs:/FileStore/tables/resultado_parquet/_started_1288630687672049619,_started_1288630687672049619,0,1729476570000
dbfs:/FileStore/tables/resultado_parquet/part-00000-tid-1288630687672049619-e6321805-1ee3-42fd-8c2f-a859be0579b8-43-1-c000.snappy.parquet,part-00000-tid-1288630687672049619-e6321805-1ee3-42fd-8c2f-a859be0579b8-43-1-c000.snappy.parquet,3876,1729476571000


In [0]:
dbfs cp -r

  File <command-949380784529352>:1
    dbfs cp -r
         ^
SyntaxError: invalid syntax
